In [1]:
import bs4
import requests
from lxml import etree
import numpy as np
import spacy
from urllib.request import urlopen
from tqdm import tqdm 
import nltk
from bs4 import Doctype, NavigableString
import json
nltk.download('punkt')

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Gibson, DOM

In [3]:
freq = dict()
hsh2str = dict()

In [4]:
def count_template_hash(soup):
  html_hsh = hash(str(soup))
  if html_hsh in freq:
    freq[html_hsh] += 1
  else:
    freq[html_hsh] = 1
    hsh2str[html_hsh] = str(soup)


  if not isinstance(soup, NavigableString):
    children = soup.findChildren(recursive=False)
    for child in children:
      count_template_hash(child)

## First Pass

In [5]:
for url_idx in tqdm(range(len(sites))):
  url = sites[url_idx]
  html = urlopen(url).read()
  soup = bs4.BeautifulSoup(html, features="html.parser")

  # removed Doctype
  for item in soup.contents:
    if isinstance(item, Doctype):
        item.extract()

  # removes script tags
  for script in soup(["script"]):
      script.extract()

  count_template_hash(soup)

100%|██████████| 63/63 [01:18<00:00,  1.24s/it]


## Second Pass

In [6]:
F = 2

def remove_templates(soup):
  html_hsh = hash(str(soup))
  if html_hsh in freq and freq[html_hsh] >= F and not soup.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
    soup.extract()
    return

  if not isinstance(soup, NavigableString):
    children = soup.findChildren(recursive=False)
    for child in children:
      remove_templates(child)

In [7]:
def get_header(soup):
  if soup == None:
    return None

  if soup.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
    return soup

  for sibling in soup.next_siblings:
      if sibling.name in ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']:
        return sibling

  return get_header(soup.findParent())

In [23]:
def get_attributes(soup):
  attribute_keywords = {'research': ['research', 'interests', 'research interest', 'research interests', 'research area', 'research areas', "research description"], 'publications': ['publications', 'papers', 'recent publications', 'recent papers'], 'education': ['education', 'alma mater'], 'students': ['students', 'graduate students', 'undergraduate students'], 'teaching': ['teaching', 'courses', 'course', 'class'], 'contact': ['contact', 'contact information', 'email']}
  attributes = {k:[] for k in attribute_keywords}

  for header_num in range(1, 7):
    header_name = f'h{header_num}'
    headers = soup.find_all(header_name)

    for header in headers:
      header_text = header.text.lower()
      for attribute in attributes:
        for keyword in attribute_keywords[attribute]:
          if header_text == keyword:
            for sibling in header.next_siblings:
              sibling_text = sibling.text
              sibling_text = sibling_text.split('\n')

              for text in sibling_text:
                if not text.isspace() and not text == "":
                  attributes[attribute].append(text)
            break

  return attributes

In [24]:
url = "https://math.illinois.edu/directory/profile/palbin"
html = urlopen(url).read()
soup = bs4.BeautifulSoup(html, features="html.parser")

# removed Doctype
for item in soup.contents:
  if isinstance(item, Doctype):
      item.extract()

# removes script tags
for script in soup(["script"]):
    script.extract()

remove_templates(soup)
attributes = get_attributes(soup)


In [25]:
print(json.dumps(attributes, indent=4))

{
    "research": [
        "My research is in geometric analysis. I am particularly interested in analytic representations of topological invariants, analysis on non-compact or singular spaces, spectral geometry, heat kernels, and Dirac operators."
    ],
    "publications": [
        ", Rochon, F., & Sher, D. (2022). A CHEEGER\u2013M\u00dcLLER THEOREM FOR MANIFOLDS WITH WEDGE SINGULARITIES. Analysis and PDE, (3), 567-642. https://doi.org/10.2140/apde.2022.15.567, & Quan, H. (2022). Sub-Riemannian Limit of the Differential Form Heat Kernels of Contact Manifolds. , (8), 5818-5881. https://doi.org/10.1093/imrn/rnaa270, Rochon, F., & Sher, D. (2021). Resolvent, heat kernel, and torsion under degeneration to fibered cusps. , (1314), 1-138. https://doi.org/10.1090/memo/1314 (2020). Poincar\u00e9-Lovelock metrics on conformally compact manifolds. , , [107108]. https://doi.org/10.1016/j.aim.2020.107108, Rochon, F., & Sher, D. (2018). Analytic torsion and R-torsion of Witt representations on 